In [1]:
import sys
sys.executable


'/Users/rose/anaconda3/bin/python'

In [2]:
import pandas as pd
import numpy as np
import numpy as np
import gc # garbage collector
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split

/Users/rose/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# read the training data
test_data = pd.read_csv('data/test.csv')
test_data.sample(5)

,click_id,ip,app,device,os,channel,click_time
559191,559191,73162,2,1,19,237,2017-11-10 04:09:57
10859823,10859823,62717,9,1,8,129,2017-11-10 10:32:16
13579950,13579950,38376,9,1,51,244,2017-11-10 13:24:21
14150525,14150525,12815,3,1,42,424,2017-11-10 13:35:07
3724655,3724655,53645,12,1,14,178,2017-11-10 05:07:18


In [4]:
# read the training data
train_data = pd.read_csv('data/train.csv', nrows=75000000)
train_data.head()


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0


In [5]:
# check the data column dtype
train_data['click_time'].head()

0    2017-11-06 14:32:21
1    2017-11-06 14:33:34
2    2017-11-06 14:34:12
3    2017-11-06 14:34:52
4    2017-11-06 14:35:08
Name: click_time, dtype: object

The data data type is object, we need to change the column to datetime. We also want to gete the specific day and hour of the click time.

In [6]:
def change_date(df, time):
    r_time = pd.to_datetime(time)
    df['hour'] = pd.to_datetime(r_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(r_time).dt.day.astype('uint8')
    return df

In [7]:
# datetime on test.data
change_date(test_data, test_data.click_time)
test_data.head()

,click_id,ip,app,device,os,channel,click_time,hour,day
0,0,5744,9,1,3,107,2017-11-10 04:00:00,4,10
1,1,119901,9,1,3,466,2017-11-10 04:00:00,4,10
2,2,72287,21,1,19,128,2017-11-10 04:00:00,4,10
3,3,78477,15,1,13,111,2017-11-10 04:00:00,4,10
4,4,123080,12,1,13,328,2017-11-10 04:00:00,4,10


In [8]:
# datetime on train_data
change_date(train_data, train_data.click_time)
train_data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,day
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0,14,6
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0,14,6
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0,14,6
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0,14,6
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0,14,6


In [9]:
most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
least_freq_hours_in_test_data = [6, 11, 15]
train_data['in_test_frq'] = (   3 
                         - 2*train_data['hour'].isin(  most_freq_hours_in_test_data ) 
                         - 1*train_data['hour'].isin( least_freq_hours_in_test_data ) ).astype('uint8')


###### Analyse the label(is_attributed) by the features

In [10]:
def group(dataset, g, x):
    gp = dataset.groupby(by=g)[['channel']].count().reset_index().rename(index=str, columns={'channel': x})
    dataset = dataset.merge(gp, on=g, how='left')
    return dataset

In [11]:
# ip-click_time_combination
x = 'ip_tcount'
g = ['ip','day','hour']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [12]:
# ip-app_os combination
x = 'ip_app_os_count'
g = ['ip','app','os']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [13]:
# ip-device_os combination
x = 'ip_device_os_count'
g = ['ip','device','os']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [14]:
# ip_is_attributed, device combination
x = 'ip_is_attributed_count'
g = ['ip', 'app', 'device', 'os']
test_data = group(test_data, g, x)
train_data = group(train_data, g, x)

In [15]:
# group by hour frequencies
x = 'hour_frequencies'
g = ['ip', 'app', 'os' , 'in_test_frq']
train_data = group(train_data, g, x)

###### using mean and variance

In [16]:
# mean hour
print('grouping by : ip_app_chl_mean_hour')
gp = train_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
train_data = train_data.merge(gp, on=['ip','app', 'channel'], how='left')
# test 
gp = test_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
test_data = test_data.merge(gp, on=['ip','app', 'channel'], how='left')

# variance hour
print('grouping by : ip_app_chl_mean_hour')
gp = train_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_var_hour'})
train_data = train_data.merge(gp, on=['ip','app', 'channel'], how='left')
# test 
gp = test_data[['ip','app', 'os', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_var_hour'})
test_data = test_data.merge(gp, on=['ip','app', 'channel'], how='left')

grouping by : ip_app_chl_mean_hour
grouping by : ip_app_chl_mean_hour


In [17]:
# split train_data into training and validation
train_data, validation_data = train_test_split(train_data,  train_size=0.75, random_state=0)
print(train_data.shape)
print(validation_data.shape)


(56250000, 18)
(18750000, 18)


In [18]:
# convert the dataset into LGBM format in this fromat:
# lgb.Dataset(data, label=label, feature_name=['c1', 'c2', 'c3'], categorical_feature=['c3'])
y = 'is_attributed'
x = [ 'app','device','os', 'channel', 'hour', 'day', 
              'ip_tcount', 'ip_app_os_count',
              'ip_device_os_count', 'ip_is_attributed_count']
categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']

l_train = lgb.Dataset(train_data[x].values, 
                    label=train_data[y].values, 
                    feature_name=x,
                    categorical_feature=categorical
                        )

l_validation = lgb.Dataset(validation_data[x].values, 
                    label=validation_data[y].values, 
                    feature_name=x,
                    categorical_feature=categorical
                        )


In [19]:
# LGBM parameters
parameters = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric':'auc',
    'learning_rate': 0.1,
    'num_leaves': 200,
    'max_depth':2,  
    'min_data_in_leaf': 1000,
    'min_child_samples': 100,
    'max_bin': 100,  
    'subsample': 0.9,  
    'subsample_freq': 3,
    'colsample_bytree': 0.9,  
    'min_child_weight': 0,  
    'min_split_gain': 1,
    'reg_alpha': 0, 
    'reg_lambda': 0,
}


In [20]:
train_data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,day,in_test_frq,ip_tcount,ip_app_os_count,ip_device_os_count,ip_is_attributed_count,hour_frequencies,ip_app_channel_mean_hour,ip_app_channel_var_hour
26122299,92872,18,1,47,439,2017-11-07 04:58:58,NaN,0,4,7,1,244,6,55,6,6,10.160000,45.622703
10190330,27001,18,3032,607,107,2017-11-07 00:15:13,NaN,0,0,7,3,270,28,31,28,8,7.943182,43.985325
17696975,14386,2,1,18,236,2017-11-07 02:28:07,NaN,0,2,7,3,9,2,12,2,1,11.000000,162.000000
52547431,112816,12,1,13,19,2017-11-07 13:39:16,NaN,0,13,7,1,52,70,489,70,15,6.166667,26.166667
28859793,186917,3,1,16,280,2017-11-07 05:49:15,NaN,0,5,7,1,18,2,12,2,1,4.666667,12.912281


In [21]:
# train the model
model = lgb.train(parameters,
                  l_train,
                  valid_sets=[l_train, l_validation],
                  valid_names=['train','valid'],
                  early_stopping_rounds=100,
                  num_boost_round=1000,
                  verbose_eval=50,
                  feval=None,
                  )

/Users/rose/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/rose/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[50]	train's auc: 0.942608	valid's auc: 0.941739
[100]	train's auc: 0.967291	valid's auc: 0.966821
[150]	train's auc: 0.969861	valid's auc: 0.969406
[200]	train's auc: 0.971133	valid's auc: 0.970631
[250]	train's auc: 0.972113	valid's auc: 0.97159
[300]	train's auc: 0.972634	valid's auc: 0.972053
[350]	train's auc: 0.972954	valid's auc: 0.972363
[400]	train's auc: 0.973271	valid's auc: 0.972644
[450]	train's auc: 0.973491	valid's auc: 0.97287
[500]	train's auc: 0.973736	valid's auc: 0.973092
[550]	train's auc: 0.973966	valid's auc: 0.973311
[600]	train's auc: 0.974175	valid's auc: 0.9735
[650]	train's auc: 0.974373	valid's auc: 0.973696
[700]	train's auc: 0.974547	valid's auc: 0.973859
[750]	train's auc: 0.974682	valid's auc: 0.973965
[800]	train's auc: 0.974815	valid's auc: 0.974082
[850]	train's auc: 0.975	valid's auc: 0.974243
[900]	train's auc: 0.975109	valid's auc: 0.974339
[950]	train's auc: 0.975182	valid's auc: 0.97

In [22]:
# predict test data
test_pred = pd.DataFrame()
test_pred['click_id'] = test_data['click_id'].astype('int')
test_pred.head()

test_pred['is_attributed'] = model.predict(test_data[x])
test_pred.to_csv('submission_fraud.csv', index=False)